## Library

In [1]:
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from gplearn.fitness import make_fitness

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LinearRegression
import pickle
from scipy import signal

In [2]:
import cufflinks as cf
cf.go_offline()

In [3]:
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
%run ..\..\Data\triangulars.ipynb
%run ..\Function.ipynb

## Functions

### Gplearn functions

In [5]:
# custom metric
def _mape(y, y_pred, w):
    diffs = np.abs(np.divide((np.maximum(0.001, y) - np.maximum(0.001, y_pred)),np.maximum(0.001, y)))
    
    return 100. * np.average(diffs, weights=w)

mape = make_fitness(_mape, greater_is_better=False)

In [6]:
cos_7 = make_function(function=CustomSC.cos_7, name='cos_7', arity=1)

sin_7 = make_function(function=CustomSC.sin_7, name='sin_7', arity=1)

cos_365 = make_function(function=CustomSC.cos_365, name='cos_365', arity=1)

sin_365 = make_function(function=CustomSC.sin_365, name='sin_365', arity=1)

abscos_14 = make_function(function=CustomSC.abscos_14, name='abscos_14', arity=1)

abssin_14 = make_function(function=CustomSC.abssin_14, name='abssin_14', arity=1)

In [7]:
def cos_7(x1): return np.cos(2*np.pi*x1/7.001447)
    
cos_7 = make_function(function=cos_7, name='cos_7', arity=1)
    
def sin_7(x1): return np.sin(2*np.pi*x1/7.001447)
    
sin_7 = make_function(function=sin_7, name='sin_7', arity=1)
    

def cos_372(x1): return np.cos(2*np.pi*x1/372.153846)
    
cos_372 = make_function(function=cos_372, name='cos_372', arity=1)
    
def sin_372(x1): return np.sin(2*np.pi*x1/372.153846)
    
sin_372 = make_function(function=sin_372, name='sin_372', arity=1)
    

def cos_91(x1): return np.cos(2*np.pi*x1/91.283019)
    
cos_91 = make_function(function=cos_91, name='cos_91', arity=1)
    
def sin_91(x1): return np.sin(2*np.pi*x1/91.283019)
    
sin_91 = make_function(function=sin_91, name='sin_91', arity=1)
    

def cos_1209(x1): return np.cos(2*np.pi*x1/1209.5)
    
cos_1209 = make_function(function=cos_1209, name='cos_1209', arity=1)
    
def sin_1209(x1): return np.sin(2*np.pi*x1/1209.5)
    
sin_1209 = make_function(function=sin_1209, name='sin_1209', arity=1)
    

def cos_345(x1): return np.cos(2*np.pi*x1/345.571429)
    
cos_345 = make_function(function=cos_345, name='cos_345', arity=1)
    
def sin_345(x1): return np.sin(2*np.pi*x1/345.571429)
    
sin_345 = make_function(function=sin_345, name='sin_345', arity=1)
    

def cos_1612(x1): return np.cos(2*np.pi*x1/1612.666667)
    
cos_1612 = make_function(function=cos_1612, name='cos_1612', arity=1)
    
def sin_1612(x1): return np.sin(2*np.pi*x1/1612.666667)
    
sin_1612 = make_function(function=sin_1612, name='sin_1612', arity=1)
    

def cos_73(x1): return np.cos(2*np.pi*x1/73.30303)
    
cos_73 = make_function(function=cos_73, name='cos_73', arity=1)
    
def sin_73(x1): return np.sin(2*np.pi*x1/73.30303)
    
sin_73 = make_function(function=sin_73, name='sin_73', arity=1)
    

def cos_45(x1): return np.cos(2*np.pi*x1/45.641509)
    
cos_45 = make_function(function=cos_45, name='cos_45', arity=1)
    
def sin_45(x1): return np.sin(2*np.pi*x1/45.641509)
    
sin_45 = make_function(function=sin_45, name='sin_45', arity=1)

In [8]:
def one(x1): return np.ones(len(x1))

one = make_function(function = one, name = 'one', arity = 1)

In [9]:
def time(x1): return np.arange(1, len(x1)+1)

time = make_function(function = time,name = 'time', arity = 1)

### model functions

In [10]:
def predict1(X, y, model):
    # scale data
    X_train1, X_test1, y_train, y_test = train_test_split(X, consumo, test_size = 365, shuffle = False)
    scaler1 = MaxAbsScaler(); scaler1.fit(y_train)
    y_train = scaler1.transform(y_train).reshape(-1) 
    y_test = scaler1.transform(y_test).reshape(-1)
    
    scaler = MaxAbsScaler(); scaler.fit(X_train1)
    X_train = scaler.transform(X_train1)
    X_test = scaler.transform(X_test1)
    # predict with loaded model (gplearn)
    y_predtr = model.predict(X_train)
    y_predte = model.predict(X_test)
    # join forecasting and train prediction
    y_pred = np.hstack([y_predtr, y_predte])
    # create dataframe
    y_pred = scaler1.inverse_transform(y_pred.reshape(-1, 1))
    plotting = pd.DataFrame(y_pred, index = consumo.index, columns = ["predict"])
    plotting["real"] = consumo.to_numpy()
    return plotting

In [11]:
def mape_comp(y):
    return np.mean(np.abs((y.real - y.predict)/y.real))*100

In [115]:
def mape_comp_res(y):
    return np.mean(np.abs((y.residual - y.predict_res)/y.residual))*100

## Create X

In [12]:
path_consumo = Path().resolve().parents[1] / "Data" / "Data1.xlsx"
consumo = pd.read_excel(path_consumo)
consumo = consumo.set_index("fecha").loc["2007-01-01":"2020-03-30"]

In [13]:
festivos=pd.read_excel(Path().resolve().parents[1] / "Data" /"Festivos.xlsx")
festivos2=pd.read_excel(Path().resolve().parents[1] / "Data" / "Festivos2.xlsx")

In [14]:
fest = triangulars().festivos(X = consumo, festivos = festivos, festivos2 = festivos2)

In [15]:
t = np.arange(1, consumo.size+1).reshape(-1, 1)
day = consumo.index.dayofyear
wend=consumo.index.weekday
wday=consumo.index.weekday
weekd = consumo.index.weekday
month = consumo.index.month

weekd1 = pd.get_dummies(weekd, prefix = "wday", drop_first = True)
weekd1.index = consumo.index

X = triangulars().diffseason(consumo)
# X1 = triangulars().diffclima(consumo)
X = pd.concat([X, weekd1], axis = 1)
X = pd.concat([X, fest], axis = 1)

X["t"] = t
X["day"] = day
X["month"] = month
X["weekd"] = weekd
X["wend"]=wend
X["wday"]=wday
X["wend"] = X["wend"].replace([0,1,2,3,4,5,6],[.5,0,0,0,.5,1,1])
X["wday"] = X["wday"].replace([0,1,2,3,4,5,6],[.5,1,1,1,.5,0,0])

In [16]:
t = np.arange(1, consumo.size+1).reshape(-1, 1)
day = consumo.index.dayofyear
X1 = triangulars().diffseason(consumo)
X1["t"] = t
X1["day"] = day

In [17]:
X2 = fest
X2["day"] = day
X2["t"] = t
X2 = pd.concat([X2, weekd1], axis = 1)

## Load model

In [18]:
path_topickle = Path().resolve().parents[1] / "Models" / "GRegressor" / "GR_gp_dwsmwds.pkl"
with open(path_topickle, 'rb') as f:
    model1 = pickle.load(f)

In [19]:
yi = predict1(X = X, y = consumo, model = model1)

In [20]:
y_residual = yi.real - yi.predict

In [21]:
# yi.iplot()

In [22]:
# y_residual.iplot()

## Fourier

In [23]:
# plt.figure()
# plt.plot(1/f, asd)

In [129]:
f, asd = signal.periodogram(y_residual, 1)
# plt.figure()
# plt.plot(1/f, asd)
picos = pd.DataFrame(asd, 1/(f), columns=["potencia"])
picos = picos.sort_values(by="potencia",ascending=False).reset_index().head(12).tail(11)
picos.columns = ["periodo", "potencia"]

# #fourier
# sencos = pd.DataFrame(index = y_residual.index)
# t = np.arange(1,len(y_residual)+1)
# sencos["t"]=t
# for i  in  picos.periodo:
#         sencos[f"{i:.2f}_sen"] = np.abs(np.sin(((2*np.pi)/(i))*t))
#         sencos[f"{i:.2f}_cos"] = np.abs(np.cos(((2*np.pi)/(i))*t))
# sencos['ones']=1
# sencos['sen1']=np.abs(np.sin(((2*np.pi)/(365.25))*t))
# sencos['cos1']=np.abs(np.cos(((2*np.pi)/(365.25))*t))
# sencos = sencos/sencos.max()

C:\Users\alber\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: RuntimeWarning:

divide by zero encountered in true_divide



In [25]:
picos

,periodo,potencia
1,7.001447,1.908232e+10
2,372.153846,9.289536e+09
3,91.283019,9.167810e+09
4,1209.500000,9.112177e+09
5,345.571429,7.706328e+09
6,1612.666667,7.156478e+09
7,73.303030,5.634094e+09
8,45.641509,5.167639e+09
9,806.333333,4.638087e+09
10,60.475000,4.363543e+09


### Create functions

In [ ]:
for i in [7.001447, 372.153846, 91.283019, 1209.5, 345.571429, 1612.666667, 73.303030, 45.641509]:
    print(f"""
def cos_{int(i)}(x1): return np.cos(2*np.pi*x1/{i})
    
cos_{int(i)} = make_function(function=cos_{int(i)}, name='cos_{int(i)}', arity=1)
    
def sin_{int(i)}(x1): return np.sin(2*np.pi*x1/{i})
    
sin_{int(i)} = make_function(function=sin_{int(i)}, name='sin_{int(i)}', arity=1)
    """)

In [12]:
for i in [7.001447, 372.153846, 91.283019, 1209.5, 345.571429, 1612.666667, 73.303030, 45.641509]:
    print(f"cos_{int(i)}, sin_{int(i)}", end = ", ")

cos_7, sin_7, cos_372, sin_372, cos_91, sin_91, cos_1209, sin_1209, cos_345, sin_345, cos_1612, sin_1612, cos_73, sin_73, cos_45, sin_45, 

## Fit the data

In [27]:
# cos_7,sin_7, cos_365, sin_365, abscos_14, abssin_14

In [145]:
model = SymbolicRegressor(population_size=2000, init_method = "half and half",
                           p_crossover=0.5, p_subtree_mutation = 0.3, p_hoist_mutation = 0.01, p_point_mutation = 0.15,
                           warm_start = False,
                           function_set=('add', 'sub', 'mul', 'div', 'sin', 'cos', 'abs', 'log', 'sqrt',
                                         cos_7, sin_7, cos_372, sin_372, cos_91, sin_91, cos_1209, sin_1209, cos_345, 
                                         sin_345, cos_1612, sin_1612, cos_73, sin_73, cos_45, sin_45,), 
#                           feature_names = [*X.columns],
#                            metric = mape,
                           max_samples = 0.95,
                           generations=100, stopping_criteria=0.01, parsimony_coefficient=0.00001,
                           verbose=1, random_state=0)

In [149]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(t, y_residual, test_size = 365, shuffle = False)


# X_train, X_test = X_train1.to_numpy(), X_test1.to_numpy()

# scaler = MaxAbsScaler(); scaler.fit(X_train1)
# X_train = scaler.transform(X_train1)
# X_test = scaler.transform(X_test1)

scaler1 = MaxAbsScaler(); scaler1.fit(y_train1.to_numpy().reshape(-1, 1))
y_train = scaler1.transform(y_train1.to_numpy().reshape(-1, 1)).reshape(-1)
y_test = scaler1.transform(y_test1.to_numpy().reshape(-1, 1)).reshape(-1)

In [150]:
# multiply by 100
# use just t
# use mae
# without max_sample

In [151]:
model.fit(X_train1, y_train) # lower persemony

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     7.06          2545.43        5         0.115875         0.120811      6.35m
   1     6.87          6711.71        5         0.115326         0.131223      6.73m
   2     6.30          29.3255        5         0.115125         0.123396      7.72m
   3     6.14          29.6679        8         0.114387         0.124207      6.23m
   4     5.57          3371.57        8         0.113729         0.136692      5.86m
   5     6.31          37.8808       11         0.113383         0.125976      5.99m
   6     8.18          3359.41       20          0.11191         0.125958      6.42m
   7    10.07          3384.23       18         0.112023         0.120571      6.77m
   8    13.64          14.9488       20          0.11067         0.120779  

  94   241.54         0.140779      252        0.0879959        0.0994979      3.60m
  95   243.63         0.128948      260        0.0879544        0.0964864      3.14m
  96   246.52         0.139706      260        0.0875823         0.103545      2.37m
  97   251.66         0.136987      275        0.0877402        0.0998586      1.50m
  98   251.28         0.137738      260        0.0877406         0.100535     44.92s
  99   253.46         0.133708      259        0.0871714         0.111073      0.00s


SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div', 'sin', 'cos', 'abs',
                                'log', 'sqrt',
                                <gplearn.functions._Function object at 0x000002D0851CE5C...
                  generations=100, init_depth=(2, 6),
                  init_method='half and half', low_memory=False,
                  max_samples=0.95, metric='mean absolute error', n_jobs=1,
                  p_crossover=0.5, p_hoist_mutation=0.01, p_point_mutation=0.15,
                  p_point_replace=0.05, p_subtree_mutation=0.3,
                  parsimony_coefficient=1e-05, population_size=2000,
                  random_state=0, stopping_criteria=0.01, tournament_size=20,
                  verbose=1, warm_start=False)

In [152]:
print(model._program)

sin_345(add(add(cos_1612(X0), add(cos_73(add(0.422, X0)), add(add(add(cos_1612(div(X0, 0.631)), add(add(cos_1612(X0), add(mul(cos_73(cos_1209(sin_1209(X0))), log(abs(0.331))), add(add(add(sin_7(add(add(add(sin_1209(X0), cos_1612(X0)), cos_1612(div(add(0.422, X0), 0.631))), add(cos_1209(abs(cos_73(cos_1612(sin_1612(-0.403))))), cos_1612(mul(X0, 0.757))))), add(mul(cos_73(sin_1612(cos_91(add(sin_372(cos_73(X0)), sin_1612(X0))))), log(sin_7(sin_91(X0)))), add(add(add(sin_7(add(add(div(sin_1209(-0.958), cos_1612(div(div(X0, 0.199), 0.631))), add(add(sin_1209(X0), cos_1612(X0)), cos_1612(div(X0, 0.631)))), add(cos_1209(abs(cos_73(cos_1612(div(X0, 0.631))))), cos_1612(mul(X0, 0.757))))), add(add(cos_1612(mul(X0, 0.631)), add(add(add(sub(cos_345(mul(X0, 0.631)), cos_7(X0)), add(add(-0.016, cos_1612(mul(X0, 0.757))), sin_1209(X0))), sin_73(0.706)), add(sin_372(X0), add(sub(sin_91(-0.068), add(sin_7(0.233), sin_91(X0))), 0.233)))), 0.496)), log(0.784)), cos_1612(mul(X0, 0.757))))), log(0.784)),

In [155]:
y_predict_tr = model.predict(X_train1)
y_predict_te = model.predict(X_test1)

In [156]:
y_prediction = np.hstack([y_predict_tr, y_predict_te])

In [157]:
y_prediction = scaler1.inverse_transform(y_prediction.reshape(-1, 1))

In [158]:
graph = pd.DataFrame(y_prediction, index = consumo.index, columns = ["predict_res"])
graph["residual"] = y_residual

In [159]:
# [["residual", "predict_res"]]

In [160]:
mape = mape_comp_res(graph)
graph[["residual", "predict_res"]].iplot(title = f"Mape: {round(mape, 2)}")

In [162]:
path_to_model = Path().resolve().parents[1] / "Models" / "GRegressor" / "gpX_gpt"
with open(path_to_model, "wb") as f:
    pickle.dump(model, f)